In [73]:
import os
import random
import numpy as np
import pandas as pd
from math import ceil
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import decomposition
from statistics import mode
#import vggish_input, vggish_slim, vggish_params, utils
#from utils import wavefile_to_waveform

In [3]:
speaker_id = ['1069','289','39','6181','8226','1334','3242','412','6272','831',
              '1553','332','5163','7402','8465','1737','3436','6000','7800',
              '8609','2159','3440','6064','7859','8770','2384','3807','6081',
              '8095','8975']

In [4]:
len(speaker_id)

30

In [14]:
# construct an df column = [emb, true_label]
a = np.array([1, 2, 3])
label_a = '1'
b = np.array([4, 5, 6])
label_b = '4'

np.concatenate((a,b), axis = 0)
#pd.DataFrame({'emb': a.T, 'label': label_a})

array([1, 2, 3, 4, 5, 6])

In [27]:
emb = np.load('/Users/ruoyuzhu/30-speakers/train/embeddings_6144/6064-56168-0016.npy')

In [32]:
len(emb[0])

6144

In [30]:
emb.shape[0] # how many split for each audio file = length
emb.shape[1] # = 6144

153

In [6]:
np.concatenate((a, b), axis=0)

array([1, 2, 3, 4, 5, 6])

In [ ]:
#train
import os

directory = r'/Users/ruoyuzhu/30-speakers/train/embeddings_6144/'


first = True
for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        
        if first:
            emb = np.load(entry.path) #load array 
            audio_size = emb.shape[0]  # how many 1s clips in each file
            true_label = entry.path.split('/')[6].split('-')[0] #replace 3 by path number
            X = emb
            y = np.repeat(true_label, audio_size)
            print(y.shape)
            print(X.shape)
            first = False
        else:
            emb = np.load(entry.path)
            audio_size = emb.shape[0]
            true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
            X = np.concatenate((X, emb))
            next_label = np.repeat(true_label, audio_size)
            y = np.concatenate((y, next_label))
            
            

        

(121,)
(121, 6144)


In [ ]:
import pandas as pd 
pd.DataFrame(X).to_csv("X.csv")
pd.DataFrame(y).to_csv("y.csv")

In [ ]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)
print('Fitting model..')

In [ ]:
#test
directory = r'/Users/ruoyuzhu/30-speakers/test/embeddings_6144/'

y_true = []
y_pred = []
first = True
for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        emb_test = np.load(entry.path)
        #audio_size = emb.shape[0]
        true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
        predict_label = mode(clf.predict(emb_test))
        
        y_true.append(true_label)
        y_pred.append(predict_label)
            


In [ ]:
# print and store the results
conf_matrix = confusion_matrix(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)

In [5]:
#DATA_FOLDER = '/Users/ruoyuzhu/sklearn-audio-transfer-learning/data/'  #'../data/'
DATA_FOLDER = '/Users/ruoyuzhu/30-speakers/
config = {
    'dataset': 'LibriSpeech-360',
    'num_classes_dataset': 30, #30 speakers
    'audio_folder': DATA_FOLDER + 'audio/GTZAN/genres/',
    'audio_paths_train': DATA_FOLDER + 'index/GTZAN/train_filtered.txt',
    'audio_paths_test': DATA_FOLDER + 'index/GTZAN/test_filtered.txt',
    'batch_size': 8, 
    'features_type': 'openl3', 
    'pca': False, 
    'model_type': 'RandomForest',
    'load_training_data': False, 
    'load_evaluation_data': False 
}

In [6]:
## import function wavefile_to_waveform
import os
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.io import wavfile


# def load_path2gt(paths_file, config):
#     """ Given the path, construct the ground truth vectors.
#         This function heavily relies on path2gt_datasets(.),
#         where the relation between the path and ground truth
#         are defined.
#     """
#     paths = list()
#     path2gt = dict()
#     path2onehot = dict() # REMOVE IF NOT USED!
#     pf = open(paths_file)
#     for path in pf.readlines():
#         path = path.rstrip('\n')
#         paths.append(path)
#         label = path2gt_datasets(path, config['dataset'])
#         path2gt[path] = label
#         path2onehot[path] = label2onehot(label, config['num_classes_dataset'])  #array([0., 0., 1., 0., 0.])
#     return paths, path2gt, path2onehot


# def label2onehot(label, num_classes):
#     """ Convert class label to one hot vector.
#         Example: label2onehot(label=2, num_classes=5) > array([0., 0., 1., 0., 0.])
#     """
#     onehot = np.zeros(num_classes)
#     onehot[label] = 1
#     return onehot



In [6]:
def matrix_visualization(matrix,title=None):
    """ Visualize 2D matrices like spectrograms or feature maps.
    """
    plt.figure()
    plt.imshow(np.flipud(matrix.T),interpolation=None)
    plt.colorbar()
    if title!=None:
        plt.title(title)
    plt.show()


# def wavefile_to_waveform(wav_file, features_type):
#     data, sr = sf.read(wav_file)
#     if features_type == 'vggish':
#         tmp_name = str(int(np.random.rand(1)*1000000)) + '.wav'
#         sf.write(tmp_name, data, sr, subtype='PCM_16')
#         sr, wav_data = wavfile.read(tmp_name)
#         os.remove(tmp_name)
#         # sr, wav_data = wavfile.read(wav_file) # as done in VGGish Audioset
#         assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
#         data = wav_data / 32768.0  # Convert to [-1.0, +1.0]
  
#     # at least one second of samples, if not repead-pad
#     src_repeat = data
#     while (src_repeat.shape[0] < sr): 
#         src_repeat = np.concatenate((src_repeat, data), axis=0)
#         data = src_repeat[:sr]

#     return data, sr

In [13]:
if __name__ == '__main__':

    # load train/test audio paths & ground truth variables
#     [paths_train, path2gt_train, path2onehot_train] = utils.load_path2gt(config['audio_paths_train'], config)
#     [paths_test, path2gt_test, path2onehot_test] = utils.load_path2gt(config['audio_paths_test'], config)
    
    [paths_train, path2gt_train, path2onehot_train] = load_path2gt(config['audio_paths_train'], config)
    [paths_test, path2gt_test, path2onehot_test] = load_path2gt(config['audio_paths_test'], config)
    
    paths_all = paths_train + paths_test
    print('Train examples: ' + str(len(paths_train)))
    print('Test examples: ' + str(len(paths_test)))
    print(config)

    if config['load_training_data']:
        print('Loading training features..')
        training_data = np.load(DATA_FOLDER + 'audio_representations/' + config['load_training_data'])
        [X, Y, IDS] = [training_data['X'], training_data['Y'], training_data['IDS']]

    else:
        print('Extracting training features..')
        [X, Y, IDS] = extract_features_wrapper(paths_train, path2gt_train, model=config['features_type'], 
                                               save_as='training_data_{}_{}'.format(config['dataset'], config['features_type']))

    print(X.shape)
    print(Y.shape)

    ### uncomment to vidualize the features
    #interval = range(0, len(X), int(len(X)/250))
    #print(interval)
    #utils.matrix_visualization(X[interval])

    if config['pca']: # for dimensionality reduction
        pca = decomposition.PCA(n_components=config['pca'], whiten=True)
        pca.fit(X)
        X = pca.transform(X)
        print("Shape after PCA: ", X.shape)

    ### uncomment to vidualize the features
    #utils.matrix_visualization(X[interval])

    print('Fitting model..')
    model = define_classification_model()
    model.fit(X, Y)

    print('Evaluating model..')

    if config['load_evaluation_data']:
        print('Loading evaluation features..')
        evaluation_data = np.load(DATA_FOLDER + 'audio_representations/' + config['load_evaluation_data'])
        [X, IDS] = [evaluation_data['X'], evaluation_data['IDS']]

    else:
        print('Extracting evaluation features..')
        [X, Y, IDS] = extract_features_wrapper(paths_test, path2gt_test, model=config['features_type'], 
                                               save_as='evaluation_data_{}_{}'.format(config['dataset'], config['features_type']))

    if config['pca']: # for dimensionality reduction
        X = pca.transform(X)
        print("Shape after PCA: ", X.shape)

    print('Predict labels on evaluation data')
    pred = model.predict(X)

    # agreggating same ID: majority voting
    y_pred = []
    y_true = []
    extension = IDS[0].split('.')[-1] # find original extension
    for pt in paths_test:
        tmp = pt.split('.')
        tmp[-1] = extension
        pt_extension = '.'.join(tmp) # fix extension
        y_pred.append(np.argmax(np.bincount(pred[np.where(IDS==pt_extension)]))) # majority voting
        y_true.append(int(path2gt_test[pt]))

    # print and store the results
    conf_matrix = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    experiments_folder = DATA_FOLDER + 'experiments/'
    if not os.path.exists(experiments_folder):
        os.makedirs(experiments_folder)
    results_file_name = 'results_{}_{}_{}_{}.txt'.format(config['dataset'],config['features_type'],config['model_type'],random.randint(0,10000))
    to = open(experiments_folder + results_file_name, 'w')
    to.write(str(config) + '\n')
    to.write(str(conf_matrix) + '\n')
    to.write('Accuracy: ' + str(acc))
    to.close()
    print(config)
    print('Confusion matrix:')
    print(conf_matrix)    
    print('Accuracy: ' + str(acc))

  0%|          | 0/56 [00:00<?, ?it/s]

Train examples: 443
Test examples: 290
{'dataset': 'GTZAN', 'num_classes_dataset': 10, 'audio_folder': '/Users/ruoyuzhu/sklearn-audio-transfer-learning/data/audio/GTZAN/genres/', 'audio_paths_train': '/Users/ruoyuzhu/sklearn-audio-transfer-learning/data/index/GTZAN/train_filtered.txt', 'audio_paths_test': '/Users/ruoyuzhu/sklearn-audio-transfer-learning/data/index/GTZAN/test_filtered.txt', 'batch_size': 8, 'features_type': 'openl3', 'pca': False, 'model_type': 'RandomForest', 'load_training_data': False, 'load_evaluation_data': False}
Extracting training features..
Instructions for updating:
Colocations handled automatically by placer.


KeyboardInterrupt: 

In [77]:
emb = np.load('/Users/ruoyuzhu/30-speakers/train/embeddings_6144/6064-56168-0016.npy')

In [46]:
emb2 = np.load('/Users/ruoyuzhu/30-speakers/train/embeddings_6144/3436-172162-0003.npy')

In [52]:
X = np.concatenate((emb, emb2))
Y = np.repeat(label, audio_size)

array([[ 1.0185577 , -0.18536666,  1.0069084 , ...,  0.58884275,
         1.2111325 ,  2.0921428 ],
       [ 1.1984097 , -0.17363879,  0.9581679 , ...,  0.5749884 ,
         1.2113444 ,  2.2201946 ],
       [ 1.127706  , -0.15025726,  0.77957195, ...,  0.51264477,
         1.1968786 ,  2.2820084 ],
       ...,
       [ 0.88844043,  0.0154366 ,  0.5938695 , ...,  0.5180066 ,
         1.1988028 ,  2.3797178 ],
       [ 0.7915392 ,  0.06348451,  0.5508625 , ...,  0.5322705 ,
         1.1990378 ,  2.5066764 ],
       [ 0.9272685 , -0.11993069,  0.5081603 , ...,  0.4983559 ,
         1.2113365 ,  2.5066237 ]], dtype=float32)

array(['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
      dtype='<U1')